# データサイエンス100本ノック（構造化データ加工編） - Python

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler # conda install -c conda-forge imbalanced-learn

df_customer = pd.read_csv("./data/customer.csv")
df_category = pd.read_csv("./data/category.csv")
df_product = pd.read_csv("./data/product.csv")
df_receipt = pd.read_csv("./data/receipt.csv")
df_store = pd.read_csv("./data/store.csv")
df_geocode = pd.read_csv("./data/geocode.csv")

C:\Users\iiats\Anaconda3\envs\Python36-gpu\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# 演習問題

---
> P-021: レシート明細データフレーム（df_receipt）に対し、件数をカウントせよ。

In [5]:
df_receipt.head()

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1258502400,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1215820800,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1265328000,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1250812800,S14025,1102,2,CS025415000050,P060102007,1,90


In [2]:
len(df_receipt)

104681

In [3]:
#もしくはshape
df_receipt.shape

(104681, 9)

---
> P-022: レシート明細データフレーム（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [4]:
len(df_receipt["customer_id"].unique())
# 参考：https://note.nkmk.me/python-pandas-value-counts/

8307

---
> P-023: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [18]:
df_receipt2 = df_receipt.groupby("store_cd").agg({"amount":"sum", "quantity":"sum"})
df_receipt2.head()

# 参考：https://qiita.com/propella/items/a9a32b878c77222630ae

,amount,quantity
store_cd,,
S12007,638761,2099
S12013,787513,2425
S12014,725167,2358
S12029,794741,2555
S12030,684402,2403


---
> P-024: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）を求め、10件表示せよ。

In [35]:
df_receipt3 = df_receipt.groupby("customer_id")
df_receipt4 = df_receipt3["sales_ymd"].max()
df_receipt4.head()

# 最も新しい売上日（sales_ymd）= 最大値
# 参考：http://ailaby.com/groupby_easy/#id1_3

customer_id
CS001113000004    20190308
CS001114000005    20190731
CS001115000010    20190405
CS001205000004    20190625
CS001205000006    20190224
Name: sales_ymd, dtype: int64

In [36]:
pd.DataFrame(df_receipt4).head(10) # DataFrame型に変換

,sales_ymd
customer_id,
CS001113000004,20190308
CS001114000005,20190731
CS001115000010,20190405
CS001205000004,20190625
CS001205000006,20190224
CS001211000025,20190322
CS001212000027,20170127
CS001212000031,20180906
CS001212000046,20170811


---
> P-025: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上日（sales_ymd）を求め、10件表示せよ。

In [37]:
df_receipt3 = df_receipt.groupby("customer_id")
df_receipt5 = df_receipt3["sales_ymd"].min()
pd.DataFrame(df_receipt5).head(10)

# 最も古い売上日（sales_ymd）= 最小値
# 参考：http://ailaby.com/groupby_easy/#id1_3

,sales_ymd
customer_id,
CS001113000004,20190308
CS001114000005,20180503
CS001115000010,20171228
CS001205000004,20170914
CS001205000006,20180207
CS001211000025,20190322
CS001212000027,20170127
CS001212000031,20180906
CS001212000046,20170811
